In [1]:
%load_ext autoreload
%autoreload 2
import sys

# instead of creating a package using setup.py or building from a docker/singularity file,
# import the sister directory of src code to be called on in notebook.
# This keeps the notebook free from code to only hold visualizations and is easier to test
# It also helps keep the state of variables clean such that cells aren't run out of order with a mysterious state
sys.path.append("..")

In [2]:
import xarray as xr
import numpy as np
import pandas as pd
import datetime as dt
import glob
import os
import matplotlib.pyplot as plt

In [3]:
save_path = "/home/aevans/nwp_bias/src/machine_learning/data/profiler_images/"

In [4]:
radiometer_data_path = (
    "/home/aevans/nysm/archive/profiler/netcdf/proc-range/2024/02/20240201.nc"
)

In [5]:
df = xr.open_mfdataset(f"{radiometer_data_path}")

In [6]:
df

<xarray.Dataset>
Dimensions:                    (time: 144, station: 17, range: 173)
Coordinates:
  * time                       (time) datetime64[ns] 2024-02-01T00:05:00 ... ...
  * station                    (station) object 'PROF_ALBA' ... 'PROF_WEBS'
  * range                      (range) int64 0 25 50 75 ... 9250 9500 9750 10000
Data variables: (12/27)
    temperature                (station, time, range) float32 dask.array<chunksize=(17, 144, 173), meta=np.ndarray>
    temperature_qc             (station, time) float32 dask.array<chunksize=(17, 144), meta=np.ndarray>
    relative_humidity          (station, time, range) float32 dask.array<chunksize=(17, 144, 173), meta=np.ndarray>
    relative_humidity_qc       (station, time) float32 dask.array<chunksize=(17, 144), meta=np.ndarray>
    vapor_density              (station, time, range) float32 dask.array<chunksize=(17, 144, 173), meta=np.ndarray>
    vapor_density_qc           (station, time) float32 dask.array<chunksize=(17, 144), meta=np.ndarray>
    ...                         ...
    u                          (station, time, range) float32 dask.array<chunksize=(17, 144, 173), meta=np.ndarray>
    v                          (station, time, range) float32 dask.array<chunksize=(17, 144, 173), meta=np.ndarray>
    w                          (station, time, range) float32 dask.array<chunksize=(17, 144, 173), meta=np.ndarray>
    velocity                   (station, time, range) float32 dask.array<chunksize=(17, 144, 173), meta=np.ndarray>
    direction                  (station, time, range) float32 dask.array<chunksize=(17, 144, 173), meta=np.ndarray>
    dewpoint                   (station, time, range) float32 dask.array<chunksize=(17, 144, 173), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.8
    title:        summarized profiler network products
    comment:      All times are in UTC. Minimal QC has been applied. Use with...
    last_update:  2024-02-02T00:51:07Z
    featureType:  timeSeries
    source:       summary calculated from original native samples from lidar ...
    institution:  NYS Mesonet at the State University of New York at Albany

In [7]:
df = df.to_dataframe()

In [8]:
df.reset_index(inplace=True)

In [9]:
# sites = df['station'].unique()
# for i in np.arange(2018,2026):
#     for s in sites:
#         os.mkdir(f'{save_path}/{i}/{s}')

In [10]:
df["temperature"] = df["temperature"] - 273.13
df["ir_temperature"] = df["ir_temperature"] - 273.13
df["dewpoint"] = df["dewpoint"] - 273.13
df["relative_humidity"] = df["relative_humidity"] / 100

In [11]:
for c in df.columns:
    print("column", c)
    # Count NaN values in the 'temperature' column
    nan_count = df[c].isna().sum()

    percent = (nan_count / 423504) * 100
    print("percent of column that is corrupted", percent)

column time
percent of column that is corrupted 0.0
column station
percent of column that is corrupted 0.0
column range
percent of column that is corrupted 0.0
column temperature
percent of column that is corrupted 17.647058823529413
column temperature_qc
percent of column that is corrupted 17.647058823529413
column relative_humidity
percent of column that is corrupted 17.647058823529413
column relative_humidity_qc
percent of column that is corrupted 17.647058823529413
column vapor_density
percent of column that is corrupted 17.647058823529413
column vapor_density_qc
percent of column that is corrupted 17.647058823529413
column liquid
percent of column that is corrupted 17.647058823529413
column liquid_qc
percent of column that is corrupted 17.647058823529413
column cloud_base
percent of column that is corrupted 17.647058823529413
column integrated_vapor
percent of column that is corrupted 17.647058823529413
column integrated_liquid
percent of column that is corrupted 17.64705882352941

In [12]:
# Drop columns with the suffix '_qc'
df = df.drop(columns=df.filter(like="_qc").columns)
df = df.drop(columns=df.filter(like="surface").columns)
drop_list = ["v", "w", "velocity", "direction", "u", "cnr", "rws"]
df = df.drop(columns=drop_list)
df = df[df["range"] <= 5000]
df.head()

,time,station,range,temperature,relative_humidity,vapor_density,liquid,cloud_base,integrated_vapor,integrated_liquid,ir_temperature,rain_flag,pressure_level,dewpoint
0,2024-02-01 00:05:00,PROF_ALBA,0,1.214966,0.992700,5.21850,0.001,0.7,1.1085,0.3445,-5.089996,0.0,1008.226013,1.100494
1,2024-02-01 00:05:00,PROF_ALBA,25,0.898499,0.948932,4.98975,0.001,0.7,1.1085,0.3445,-5.089996,0.0,1005.093445,0.464630
2,2024-02-01 00:05:00,PROF_ALBA,50,0.582001,0.905165,4.76100,0.001,0.7,1.1085,0.3445,-5.089996,0.0,1001.960876,-0.196716
3,2024-02-01 00:05:00,PROF_ALBA,75,0.350769,0.901927,4.65275,0.001,0.7,1.1085,0.3445,-5.089996,0.0,998.841553,-0.523346
4,2024-02-01 00:05:00,PROF_ALBA,100,0.119507,0.898690,4.54450,0.001,0.7,1.1085,0.3445,-5.089996,0.0,995.722229,-0.856415


In [13]:
df = df[df["station"] == "PROF_ALBA"]
df.fillna(-999, inplace=True)
df = df.drop(columns="station")
df["time"] = pd.to_datetime(df["time"])

In [14]:
# Extract unique days
unique_days = df["time"].dt.date.unique()

In [15]:
unique_days

array([datetime.date(2024, 2, 1)], dtype=object)

In [16]:
# # Filter rows where the hour matches the query time's hour
# filtered_df = df[df['datetime'].dt.hour == query_time.hour]

In [17]:
df

,time,range,temperature,relative_humidity,vapor_density,liquid,cloud_base,integrated_vapor,integrated_liquid,ir_temperature,rain_flag,pressure_level,dewpoint
0,2024-02-01 00:05:00,0,1.214966,0.992700,5.21850,0.0010,0.700000,1.108500,0.344500,-5.089996,0.0,1008.226013,1.100494
1,2024-02-01 00:05:00,25,0.898499,0.948932,4.98975,0.0010,0.700000,1.108500,0.344500,-5.089996,0.0,1005.093445,0.464630
2,2024-02-01 00:05:00,50,0.582001,0.905165,4.76100,0.0010,0.700000,1.108500,0.344500,-5.089996,0.0,1001.960876,-0.196716
3,2024-02-01 00:05:00,75,0.350769,0.901927,4.65275,0.0010,0.700000,1.108500,0.344500,-5.089996,0.0,998.841553,-0.523346
4,2024-02-01 00:05:00,100,0.119507,0.898690,4.54450,0.0010,0.700000,1.108500,0.344500,-5.089996,0.0,995.722229,-0.856415
...,...,...,...,...,...,...,...,...,...,...,...,...,...
420679,2024-02-01 23:55:00,4800,-30.447113,0.515913,0.47950,0.0255,0.766667,1.191667,0.351667,-4.785004,0.0,528.226868,-29.268570
420680,2024-02-01 23:55:00,4850,-30.685211,0.495881,0.45400,0.0250,0.766667,1.191667,0.351667,-4.785004,0.0,524.552917,-29.854279
420681,2024-02-01 23:55:00,4900,-30.923309,0.475849,0.42850,0.0245,0.766667,1.191667,0.351667,-4.785004,0.0,520.878967,-30.469666
420682,2024-02-01 23:55:00,4950,-31.161407,0.455817,0.40300,0.0240,0.766667,1.191667,0.351667,-4.785004,0.0,517.205017,-31.118240


In [18]:
skip_list = ["time", "range"]
for c in df.columns:
    if c in skip_list:
        continue
    else:
        var_pivot = df.pivot(index="range", columns="time", values=c)
        var_array = var_pivot.to_numpy()
        try:
            stacked_array = np.vstack((stacked_array, var_array))
        except:
            stacked_array = var_array

In [19]:
stacked_array

array([[  1.2149658 ,   1.2149658 ,   1.1199951 , ...,   2.1774902 ,
          2.1600037 ,   2.2399902 ],
       [  0.89849854,   0.8402405 ,   0.86260986, ...,   1.8616333 ,
          1.7894897 ,   1.8252563 ],
       [  0.58200073,   0.46551514,   0.6052551 , ...,   1.5457764 ,
          1.4190063 ,   1.4105225 ],
       ...,
       [-30.804703  , -30.691391  , -31.263977  , ..., -30.182663  ,
        -30.291306  , -30.469666  ],
       [-31.357376  , -31.252213  , -31.80751   , ..., -30.7677    ,
        -30.893295  , -31.11824   ],
       [-31.936768  , -31.840546  , -32.376984  , ..., -31.38266   ,
        -31.5271    , -31.804108  ]], dtype=float32)